In [ ]:
import os
import sklearn_crfsuite
from sklearn_crfsuite import metrics

file_path = "spacing_data.txt"

with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()
    
# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []
for line in lines:
    pieces = line.strip().split("\t")
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence, label))

number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]

print("train개수: ", len(train_datas))
print("test개수: ", len(test_datas))

In [ ]:
# 데이터 변환
def sent2feature(eumjeol_sequence):
    features = []
    sequence_length = len(eumjeol_sequence)
    for index, eumjeol in enumerate(eumjeol_sequence):
        feature = {"BOS": False, "EOS": False, "WORD": eumjeol, "IS_DIGIT": eumjeol.isdigit()}
        
        if(index == 0):
            feature["BOS"] = True
        elif(index == sequence_length-1):
            feature["EOS"] = True
        
        if(index-1 <= sequence_length-1):
            feature["-1_WORD"] = eumjeol_sequence[index-1]
        if(index-2 <= sequence_length-1):
            feature["-2_WORD"] = eumjeol_sequence[index-2]
            
        if(index+1 <= sequence_length-1):
            feature["+1_WORD"] = eumjeol_sequence[index+1]
        if(index+2 <= sequence_length-1):
            feature["+2_WORD"] = eumjeol_sequence[index+2]
        
        features.append(feature)
    
    return features

In [ ]:
# 데이터 생성
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

In [ ]:
# CRFs 학습
crf = sklearn_crfsuite.CRF()
crf.fit(train_x, train_y)

In [ ]:
# CRFs 평가

def show_predict_result(test_datas, predict):
    for index_1 in range(len(test_datas)):
        eumjeol_sequence, correct_labels = test_datas[index_1]
        predict_labels = predict[index_1]
        
        correct_sentence, predict_sentence = "", ""
        for index_2 in range(len(eumjeol_sequence)):
            if(index_2==0):
                correct_sentence += eumjeol_sequence[index_2]
                predict_sentence += eumjeol_sequence[index_2]
                continue
            
            if(correct_labels[index_2]=="B"):
                correct_labels += " "
            correct_sentence += eumjeol_sequence[index_2]
            
            if(predict_labels[index_2] == "B"):
                predict_sentence += " "
            predict_sentence += eumjeol_sequence[index_2]
        
        print("정답 문장: " + correct_sentence)
        print("출력 문장: " + predict_sentence)
        print()
    
    predict = crf.predict(test_x)
    print("ACC Score"+ str(metrics.flat_accuracy_score(test_y, predict)))
    print()
    print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
    print()
    
    show_predict_result(test_datas[:10], predict[:10])